In [297]:
import json
import os
import re
import google.generativeai as genai

import numpy as np
import ollama
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import traceback

import yfinance as yf
from datetime import datetime, timedelta
from newsapi import NewsApiClient
from fredapi import Fred
from sec_api import QueryApi
import requests

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.chains import RetrievalQA
from langchain_text_splitters import RecursiveCharacterTextSplitter # Split texts into manageble chunks
from langchain_classic.embeddings import HuggingFaceEmbeddings # Converts text chunk into numerical vector
from langchain_classic.vectorstores import FAISS # Helps in finding the similarity between texts in the question and the document
from langchain_core.documents import Document

## Load Environment Variables

In [298]:
def load_env(filepath="config/aai_520_proj.config"):
    """
    Loads environment variables from the aai_520_project.config.
    Each line in the file should be in the format KEY=VALUE.
    """
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    os.environ[key] = value
        print(f"Environment variables loaded from {filepath}")
    except FileNotFoundError:
        print(f"Error: Config file not found at {filepath}. Make sure it is in the project config directory.")
    except Exception as e:
        print(f"Error loading environment variables from {filepath}: {e}")


## RAG Pipeline

In [299]:
class RAG_pipeline:
    def __init__(self,modelName, apiKey):
        self.modelName = modelName
        self.apiKey = apiKey

    def getLLM(self):
        # "gemini-2.5-flash"
        llm = ChatGoogleGenerativeAI(model=self.modelName, google_api_key=os.getenv(self.apiKey))
        return llm

    def getLLM_withlayers(self, context, prompt):
        content = ""
        # print(context)
        for i in context:
            # print(i)
            if(context[i]):
                content+=(i+":\n")
                content+=(str(context[i])+"\n")
        
        print(content)
        docs = [Document(page_content=content)]
        
        # 2. Split docs into smaller chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.split_documents(docs)
        
        # 3. Create embeddings
        embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        # embeddings = [embedding_model.embed(chunk.page_content) for chunk in chunks]
        
        # 4. Store in vector database (FAISS)
        vector_store = FAISS.from_documents(chunks, embedding_model)

        return vector_store

## LLM Integration

In [300]:
def call_gemini(system_instruction: str, user_prompt: str, json_output: bool = True) -> dict | str:
    """
    Calls the Gemini API with a system instruction and user prompt.

    Args:
        system_instruction: The system instruction for the model.
        user_prompt: The user's prompt.
        json_output: Whether to expect a JSON output from the model.

    Returns:
        A dictionary if json_output is True, otherwise a string.
    """


    genai.configure(
        api_key=os.environ.get('GOOGLE_API_KEY'),
    )
    
    model = genai.GenerativeModel(
        model_name=os.environ.get('GEMINI_MODEL_NAME'),
        generation_config={"response_mime_type": "application/json"} if json_output else None
    )
    
    prompt = f"{system_instruction}\n\n{user_prompt}"

    try:
        response = model.generate_content(prompt)
        if json_output:
            return json.loads(response.text)
        return response.text
    except Exception as e:
        print(f"An error occurred in call_gemini: {e}")
        return None
    
def call_judge_gemini(system_instruction: str, user_prompt: str, json_output: bool = True) -> dict | str:
    """
    Calls the Gemini API with a system instruction and user prompt for the judge model.

    Args:
        system_instruction: The system instruction for the model.
        user_prompt: The user's prompt.
        json_output: Whether to expect a JSON output from the model.

    Returns:
        A dictionary if json_output is True, otherwise a string.
    """

    genai.configure(
        api_key=os.environ.get('GOOGLE_API_KEY'),
    )
    
    model = genai.GenerativeModel(
        model_name=os.environ.get('JUDGE_MODEL_NAME'),
        generation_config={"response_mime_type": "application/json"} if json_output else None
    )
    
    prompt = f"{system_instruction}\n\n{user_prompt}"

    try:
        response = model.generate_content(prompt)
        if json_output:
            return json.loads(response.text)
        return response.text
    except Exception as e:
        print(f"An error occurred in call_judge_gemini: {e}")
        return None


In [301]:
class AgentLogger:
    def __init__(self, state):
        self.state = state
        self.state.setdefault("conversation_logs", [])

    def log(self, sender, receiver, content, **metadata):
        self.state["conversation_logs"].append({
            "timestamp": datetime.utcnow().isoformat(),
            "sender": sender,
            "receiver": receiver,
            "content": content,
            "metadata": metadata or {}
        })

## Memory Agent

In [302]:
class MemoryAgent:
    def __init__(self, db_path='memory_db.json'):
        self.db_path = db_path
        self.memory = self._load_memory()

    # ------------------------------------------------------------
    # Internal helper to attach logger
    # ------------------------------------------------------------
    def _get_logger(self, state):
        return AgentLogger(state) if state and "conversation_logs" in state else None

    # ------------------------------------------------------------
    # Load memory from JSON file
    # ------------------------------------------------------------
    def _load_memory(self):
        try:
            if not os.path.exists(self.db_path):
                return {}
            with open(self.db_path, 'r') as f:
                return json.load(f)
        except Exception as e:
            print(f" Failed to load memory DB: {e}")
            return {}

    # ------------------------------------------------------------
    # Save memory to disk
    # ------------------------------------------------------------
    def _save_memory(self):
        try:
            with open(self.db_path, 'w') as f:
                json.dump(self.memory, f, indent=4)
        except Exception as e:
            print(f" Failed to save memory DB: {e}")

    # ------------------------------------------------------------
    # Retrieve stored memory
    # ------------------------------------------------------------
    def retrieve(self, symbol: str, state: dict = None) -> dict:
        """Retrieves memory for a given stock symbol."""
        logger = self._get_logger(state)
        try:
            memory_entry = self.memory.get(symbol)
            if memory_entry:
                if logger:
                    logger.log("MemoryAgent", "System", f"Retrieved memory for {symbol}")
                return memory_entry
            else:
                if logger:
                    logger.log("MemoryAgent", "System", f"No memory found for {symbol}")
                return None
        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("MemoryAgent", "System",
                           f"Error retrieving memory for {symbol}: {e}",
                           level="error", traceback=error_details)
            return None

    # ------------------------------------------------------------
    # Update memory
    # ------------------------------------------------------------
    def update(self, symbol: str, final_analysis: dict, state: dict = None):
        """Updates or creates a memory entry for a stock symbol."""
        logger = self._get_logger(state)

        try:
            if symbol not in self.memory:
                self.memory[symbol] = {}

            self.memory[symbol] = {
                'summary': final_analysis.get('summary', ''),
                'key_metrics': final_analysis.get('key_metrics', {}),
                'date': datetime.now().isoformat()
            }

            self._save_memory()

            msg = f"Memory updated for {symbol}"
            print(msg)
            if logger:
                logger.log("MemoryAgent", "System", msg, payload=self.memory[symbol])

        except Exception as e:
            error_details = traceback.format_exc()
            print(f" Error updating memory for {symbol}: {e}")
            if logger:
                logger.log("MemoryAgent", "System",
                           f"Error updating memory for {symbol}: {e}",
                           level="error", traceback=error_details)



## Financial Services Planning Agent

In [303]:
class PlanningAgent:
    def __init__(self):
        pass

    def generate_plan(self, symbol: str, state: dict, memory: str = None) -> list[str]:
        """Generates a research plan for a given stock symbol."""
        
        logger = AgentLogger(state)
 
        system_instruction = (
            "You are an expert investment analyst planning a research workflow. "
            "Given the stock symbol and the historical memory, generate a list of the 5-7 most critical steps "
            "(including tool calls and internal processes) to generate a final investment thesis. "
            "Output must be a JSON array of strings."
        )
        
        user_prompt = f"Stock Symbol: {symbol}"
        if memory:
            user_prompt += f"\n\nHistorical Memory:\n{memory}"
            
        # Log outgoing LLM request
        logger.log("PlanningAgent", "LLM", f"Requesting research plan for {symbol}...", prompt=user_prompt)
        # response = call_gemini(system_instruction, user_prompt, json_output=True)

        ragObject = RAG_pipeline("gemini-2.0-flash", "GOOGLE_API_KEY")
        chain = ragObject.getLLM()
        
        response = chain.invoke(system_instruction+user_prompt).content
        
        cleaned = response.strip("```json").strip("```").strip()
        response = json.loads(cleaned)

        if response and isinstance(response, list):
            logger.log("LLM", "PlanningAgent", f"Received plan: {response}")
            return response
        else:
            logger.log("PlanningAgent", "LLM", f"Invalid or empty response for {symbol}", level="error") 
            print("Failed to generate a valid plan.")
            return []


## Prompt Chaining Agent

In [304]:
class PromptChainingAgent:
    def __init__(self):
        pass

    def _get_logger(self, state):
        """Attach logger to agent if available."""
        return AgentLogger(state) if state and "conversation_logs" in state else None

    def run(self, raw_text: str, state: dict = None) -> dict:
        """Runs a 5-stage prompt chain to process raw text with detailed logging."""

        logger = self._get_logger(state)
        results = {}

        try:
            # --------------------------------------------------------------------------------
            # Stage 1: Ingest / Preprocess
            # --------------------------------------------------------------------------------
            preprocess_prompt = f"Clean the following text and remove any boilerplate content:\n\n{raw_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 1: Preprocessing text input.")
            clean_text = call_gemini("You are a text cleaning assistant.", preprocess_prompt, json_output=False)

            if not clean_text:
                msg = "Failed to clean text."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Text cleaned successfully.", payload={"clean_text": clean_text[:500]})
            print("\n--- Cleaned Text ---")
            print(clean_text)

            # --------------------------------------------------------------------------------
            # Stage 2: Classification
            # --------------------------------------------------------------------------------
            classify_prompt = f"What is the primary event type in this text? (e.g., Earnings, Product Launch, Regulation, Macro):\n\n{clean_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 2: Classifying text.")
            classification = call_gemini("You are a text classification specialist.", classify_prompt, json_output=False)

            if not classification:
                msg = "Failed to classify text."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Classification complete.", payload={"classification": classification.strip()})
            print(f"\n--- Classification ---\n{classification}")
            results["classification"] = classification.strip()

            # --------------------------------------------------------------------------------
            # Stage 3: Extraction
            # --------------------------------------------------------------------------------
            extract_prompt = f"Extract all numerical data points (e.g., EPS, Revenue, Guidance) mentioned in the text:\n\n{clean_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 3: Extracting numerical data.")
            extracted_data = call_gemini("You are a data extraction expert.", extract_prompt, json_output=True)

            if not extracted_data:
                msg = "Failed to extract data."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Data extraction complete.", payload={"extracted_data": extracted_data})
            print(f"\n--- Extracted Data ---\n{extracted_data}")
            results["extracted_data"] = extracted_data

            # --------------------------------------------------------------------------------
            # Stage 4: Summarization
            # --------------------------------------------------------------------------------
            summarize_prompt = f"Write a concise, abstractive summary of the key market takeaway (1-2 sentences):\n\n{clean_text}"
            if logger:
                logger.log("PromptChainingAgent", "System", "Stage 4: Summarizing content.")
            summary = call_gemini("You are a financial news summarizer.", summarize_prompt, json_output=False)

            if not summary:
                msg = "Failed to summarize text."
                if logger:
                    logger.log("PromptChainingAgent", "System", msg, level="error")
                return {"error": msg}

            if logger:
                logger.log("PromptChainingAgent", "System", "Summary complete.", payload={"summary": summary.strip()})
            print(f"\n--- Summary ---\n{summary}")
            results["summary"] = summary.strip()

            # --------------------------------------------------------------------------------
            # Final Results
            # --------------------------------------------------------------------------------
            if logger:
                logger.log("PromptChainingAgent", "System", "Prompt chaining complete.", payload=results)
            return results

        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("PromptChainingAgent", "System",
                           f"Unhandled exception in prompt chain: {e}",
                           level="error",
                           traceback=error_details)
            return {"error": f"Unhandled exception: {e}"}



## Routing Agent

In [305]:
class RoutingAgent:
    def __init__(self):
        pass

    def _get_logger(self, state):
        """Attach logger if conversation state is provided."""
        return AgentLogger(state) if state and "conversation_logs" in state else None

    def route(self, classification: str, state: dict = None) -> str:
        """Determines the next agent path based on classification with structured logging."""
        logger = self._get_logger(state)

        try:
            if not classification or not isinstance(classification, str):
                msg = "Invalid or empty classification received."
                if logger:
                    logger.log("RoutingAgent", "System", msg, level="error")
                return "GeneralAnalysis"

            normalized_class = classification.lower().strip()
            if logger:
                logger.log(
                    "RoutingAgent",
                    "System",
                    f"Received classification: '{classification}'",
                    payload={"normalized_class": normalized_class}
                )

            if 'earnings' in normalized_class:
                route = 'EarningsModelRun'
            elif 'regulation' in normalized_class:
                route = 'ComplianceCheck'
            elif 'product launch' in normalized_class or 'launch' in normalized_class:
                route = 'MarketImpactAnalysis'
            else:
                route = 'GeneralAnalysis'

            if logger:
                logger.log(
                    "RoutingAgent",
                    "System",
                    f"Routing decision: {route}",
                    payload={"classification": classification, "next_route": route}
                )

            return route

        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("RoutingAgent", "System",
                           f"Routing error: {e}",
                           level="error",
                           traceback=error_details)
            return "GeneralAnalysis"



## Toolbox Agent

In [306]:
class ToolboxAgent:
    def __init__(self):
        self.cache = {}
        self.newsapi = NewsApiClient(api_key=os.environ.get('NEWS_API_KEY'))
        self.fred = Fred(api_key=os.environ.get('FRED_API_KEY'))
        self.sec = QueryApi(api_key=os.environ.get('SEC_API_KEY'))

    def _is_cache_valid(self, symbol, tool_name):
        if symbol in self.cache and tool_name in self.cache[symbol]:
            timestamp = self.cache[symbol][tool_name]['timestamp']
            if datetime.now() - timestamp < timedelta(hours=24):
                return True
        return False

    # Helper to initialize logger only once per symbol/session
    def _get_logger(self, state):
        return AgentLogger(state)

    # -----------------------------------------------------------------------------------
    # YFinance Data
    # -----------------------------------------------------------------------------------
    def get_yahoo_finance_data(self, symbol: str, state: dict) -> dict:
        """Fetches price, P/E, and fundamental metrics from Yahoo Finance."""
        tool_name = 'yfinance'
        logger = self._get_logger(state)

        if self._is_cache_valid(symbol, tool_name):
            print(f"Returning cached data for {symbol} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Cache hit for {symbol}")
            return self.cache[symbol][tool_name]['data']

        try:
            print(f"Fetching data for {symbol} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Fetching Yahoo Finance data for {symbol}")
            ticker = yf.Ticker(symbol)
            info = ticker.info

            if symbol not in self.cache:
                self.cache[symbol] = {}
            self.cache[symbol][tool_name] = {
                'timestamp': datetime.now(),
                'data': info
            }

            logger.log(tool_name, "ToolboxAgent", f"Successfully fetched data for {symbol}")
            return info
        except Exception as e:
            error_details = traceback.format_exc()
            logger.log("ToolboxAgent", tool_name, f"Error fetching yfinance data for {symbol}: {e}", level="error", traceback=error_details)
            print(f" YFinance Error for {symbol}: {e}")
            return None

    # -----------------------------------------------------------------------------------
    # Financial News
    # -----------------------------------------------------------------------------------
    def get_financial_news(self, symbol: str, state: dict) -> dict:
        """Fetches financial news for a given symbol."""
        tool_name = 'newsapi'
        logger = self._get_logger(state)

        if self._is_cache_valid(symbol, tool_name):
            print(f"Returning cached news for {symbol}")
            logger.log("ToolboxAgent", tool_name, f"Cache hit for {symbol} (news)")
            return self.cache[symbol][tool_name]['data']

        try:
            print(f"Fetching news for {symbol}")
            logger.log("ToolboxAgent", tool_name, f"Fetching news for {symbol}")
            all_articles = self.newsapi.get_everything(
                q=symbol,
                language='en',
                sort_by='relevancy',
                page_size=5
            )
            if symbol not in self.cache:
                self.cache[symbol] = {}
            self.cache[symbol][tool_name] = {
                'timestamp': datetime.now(),
                'data': all_articles
            }

            logger.log(tool_name, "ToolboxAgent", f"Fetched {len(all_articles.get('articles', []))} news articles for {symbol}")
            return all_articles

        except Exception as e:
            error_details = traceback.format_exc()
            print(f" NewsAPI Error for {symbol}: {e}")
            logger.log("ToolboxAgent", tool_name, f"Error fetching news for {symbol}: {e}", level="error", traceback=error_details)
            return None

    # -----------------------------------------------------------------------------------
    # Economic Data
    # -----------------------------------------------------------------------------------
    def get_economic_data(self, indicator: str, state: dict) -> dict:
        """Fetches economic data from FRED."""
        tool_name = 'fred'
        logger = self._get_logger(state)
        if self._is_cache_valid(indicator, tool_name):
            print(f"Returning cached data for {indicator} from {tool_name}")
            return self.cache[indicator][tool_name]['data']

        try:
            print(f"Fetching data for {indicator} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Fetching economic data for indicator '{indicator}'")
            data = self.fred.get_series(indicator)

            logger.log(tool_name, "ToolboxAgent", f"Successfully fetched {len(data)} records for {indicator}")

            if indicator not in self.cache:
                self.cache[indicator] = {}
            self.cache[indicator][tool_name] = {
                'timestamp': datetime.now(),
                'data': data.to_dict()
            }
            return data.to_dict()
        except Exception as e:
            error_details = traceback.format_exc()
            print(f"An error occurred with FRED for indicator {indicator}: {e}")
            logger.log("ToolboxAgent", tool_name, f"Error fetching FRED data for {indicator}: {e}", level="error", traceback=error_details)
            return None

    # -----------------------------------------------------------------------------------
    # Filing Data (SEC EDGAR)
    # -----------------------------------------------------------------------------------
    def get_filing_data(self, indicator: str, state: dict) -> dict:
        """Fetches Filings data from Sec Edgar."""
        tool_name = 'secEdgar'
        logger = self._get_logger(state)

        query = {
            "query": (
                f'(formType:"10-K" OR formType:"10-Q" OR formType:"8-K" OR '
                f'formType:"SC 13D" OR formType:"SC 13G") AND ticker:{indicator}'
            ),
            "from": 0,
            "size": 4,
            "sort": [{"filedAt": {"order": "desc"}}]
        }
        print(query)

        logger.log("ToolboxAgent", tool_name, f"Preparing SEC EDGAR query for {indicator}", query=query)

        # Check cache first
        if self._is_cache_valid(indicator, tool_name):
            print(f"Returning cached data for {indicator} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Cache hit for SEC EDGAR filings of {indicator}")
            return self.cache[indicator][tool_name]['data']

        try:
            print(f"Fetching data for {indicator} from {tool_name}")
            logger.log("ToolboxAgent", tool_name, f"Fetching latest SEC filings for {indicator}")
            data = self.sec.get_filings(query)["filings"]

            filingDataRaw = {}
            folder_path = os.path.join("..", "utils", "filingDocuments", indicator)
            os.makedirs(folder_path, exist_ok=True)

            for filing in data:
                folder_path = "..\\utils\\filingDocuments\\"+(indicator)
                os.makedirs(folder_path, exist_ok=True)
                form_type = filing["formType"].replace("/", "-")
                description = filing["description"].replace("/", "-")
                
                fileType = {}

                for doc in filing.get("documentFormatFiles", []):
                    doc_url = doc.get("documentUrl", "")
                    if not doc_url:
                        continue

                    file_ext = os.path.splitext(doc_url)[1]
                    file_name = f"{form_type}-{description}{file_ext}"
                    file_path = os.path.join(folder_path, file_name)

                    if file_ext in [".txt", ".htm", ".html"]:
                        try:
                            response = requests.get(doc_url, timeout=10)
                            response.raise_for_status()
                            with open(file_path, "wb") as f:
                                f.write(response.content)
                            filingDataRaw[file_name] = response.content.decode("utf-8", errors="ignore")

                            logger.log(tool_name, "ToolboxAgent", f"Saved filing {file_name} for {indicator}")
                        except Exception as e:
                            error_details = traceback.format_exc()
                            logger.log("ToolboxAgent", tool_name,
                                       f"Error downloading {file_name} for {indicator}: {e}",
                                       level="error", traceback=error_details)
                    else:
                        logger.log("ToolboxAgent", tool_name, f"Skipping unsupported file type: {file_ext}")

            
            # Update cache after successful fetch
            if indicator not in self.cache:
                self.cache[indicator] = {}
            self.cache[indicator][tool_name] = {
                'timestamp': datetime.now(),
                'data': filingDataRaw
            }
            logger.log(tool_name, "ToolboxAgent", f"Fetched and cached {len(filingDataRaw)} filings for {indicator}")
            return filingDataRaw
        except Exception as e:
            error_details = traceback.format_exc()
            print(f" SEC EDGAR Error for {indicator}: {e}")
            logger.log("ToolboxAgent", tool_name,
                       f"Error fetching filings for {indicator}: {e}",
                       level="error", traceback=error_details)
            return None

    def fetch(self, tool_name: str, symbol: str, state: dict) -> dict:
        """Dynamically dispatches to the correct tool wrapper."""
        logger = self._get_logger(state)
        if tool_name == 'yfinance':
            return self.get_yahoo_finance_data(symbol, state)
        elif tool_name == 'newsapi':
            return self.get_financial_news(symbol, state)
        elif tool_name == 'fred':
            return self.get_economic_data(symbol, state)
        elif tool_name == 'secEdgar':
            return self.get_filing_data(symbol, state)
        else:
            print(f"Tool {tool_name} not recognized.")
            logger.log(tool_name, "ToolboxAgent", f"Tool {tool_name} not recognized")
            return None


## Evaluator Agent

In [307]:
class MultiAgentEvaluator:
    def __init__(self):
        self.openai_model = "gpt-4o"
        self.ollama_model = "llama2"
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")

        # Initialize OpenAI client only if key is set
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key and OpenAI is not None:
            try:
                self.client = OpenAI(api_key=api_key)
                self.mode = "openai"
            except Exception:
                self.client = None
                self.mode = "ollama"
        else:
            self.client = None
            self.mode = "ollama"

        print(f"Evaluator initialized in {self.mode.upper()} mode")

    def llm_grade(self, thesis: str, reference: str = None) -> dict:
        """Evaluate investment thesis quality using OpenAI or Ollama."""
        prompt = f"""
        Evaluate this investment thesis for clarity, factual accuracy, and rigor.
        Rate each dimension from 1–10 and summarize with justification.

        Thesis:
        {thesis}

        Reference (if provided):
        {reference}
        """

        # --- Try OpenAI first ---
        if self.mode == "openai" and self.client is not None:
            try:
                response = self.client.chat.completions.create(
                    model=self.openai_model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.2,
                )
                return {"source": "openai", "raw": response.choices[0].message.content}
            except Exception as e:
                print(f"[OpenAI Error] {e} — Falling back to Ollama.")
                self.mode = "ollama"

        # --- Fallback to Ollama ---
        if ollama is None:
            return {"error": "Neither OpenAI nor Ollama available."}

        try:
            response = ollama.chat(
                model=self.ollama_model,
                messages=[{"role": "user", "content": prompt}],
            )
            return {"source": "ollama", "raw": response["message"]["content"]}
        except Exception as e:
            return {"error": f"Both evaluators failed: {e}"}

    def embedding_consistency(self, thesis_a: str, thesis_b: str) -> float:
        """Measure semantic similarity between two analyses."""
        embeddings = self.embedder.encode([thesis_a, thesis_b])
        return cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

    def coordination_efficiency(self, logs: list) -> dict:
        """Analyze inter-agent message structure."""
        n_messages = len(logs)
        avg_message_len = np.mean([len(m["content"]) for m in logs])
        return {"n_messages": n_messages, "avg_message_len": avg_message_len}

## Final Thesis Optimizer Agent

In [308]:
class EvaluatorOptimizerAgent:
    def __init__(self):
        pass

    def _get_logger(self, state):
        """Attach logger to agent if state has conversation logs."""
        return AgentLogger(state) if state and "conversation_logs" in state else None

    def run(self, data: dict, state: dict = None) -> str:
        """Runs the evaluator-optimizer workflow with detailed logging."""

        logger = self._get_logger(state)
        try:
            # --------------------------------------------------------------------------------
            # 1. Optimizer Stage — Draft Thesis
            # --------------------------------------------------------------------------------
            draft_prompt = (
                "Generate a comprehensive draft investment analysis and thesis "
                "(Buy/Hold/Sell) based on the following data.\n\nData:\n"
                f"{data}"
            )
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Stage 1: Generating initial draft thesis.")
    
            ragObject = RAG_pipeline("gemini-2.0-flash", "GOOGLE_API_KEY")
            dbVector = ragObject.getLLM_withlayers(data, draft_prompt)
            
            docs = dbVector.similarity_search("Invetsment Outlook", k=5)
            st = "\n\n".join([doc.page_content for doc in docs])

            draft_prompt = (
                "Generate a comprehensive draft investment analysis and thesis "
                "(Buy/Hold/Sell) based on the following data.\n\nData:\n"
                f"{st}"
            )
            draft = call_gemini("You are a financial analyst drafting an investment thesis.", draft_prompt, json_output=False)
            
            if not draft:
                msg = "Failed to generate a draft."
                if logger:
                    logger.log("EvaluatorOptimizerAgent", "System", msg, level="error")
                return msg

            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Draft thesis generated successfully.",
                           payload={"draft": draft[:500]})
            print("\n--- Initial Draft ---")
            print(draft)

            # --------------------------------------------------------------------------------
            # 2. Evaluator Stage — Critique Draft
            # --------------------------------------------------------------------------------
            evaluator_prompt = (
                "Critique the following investment draft for two things:\n"
                "1. Factual consistency (do the numbers match the source data?)\n"
                "2. Logical consistency (is the 'Buy' recommendation justified by the identified risks?).\n"
                "Provide a specific suggestion for refinement.\n\n"
                f"Draft:\n{draft}"
            )
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Stage 2: Evaluating draft for consistency and logic.")
            critique = call_gemini("You are a meticulous financial evaluator.", evaluator_prompt, json_output=False)

            if not critique:
                msg = "Failed to generate a critique."
                if logger:
                    logger.log("EvaluatorOptimizerAgent", "System", msg, level="error")
                return msg

            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Critique generated successfully.",
                           payload={"critique": critique[:500]})
            print("\n--- Critique ---")
            print(critique)

            # --------------------------------------------------------------------------------
            # 3. Optimizer Stage — Refinement
            # --------------------------------------------------------------------------------
            refinement_prompt = (
                "Based on the critique provided, refine and correct the initial draft. "
                "Produce the final, polished investment thesis.\n\n"
                f"Initial Draft:\n{draft}\n\nCritique:\n{critique}"
            )
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Stage 3: Refining draft based on critique.")
            final_thesis = call_gemini("You are a financial analyst refining your work.", refinement_prompt, json_output=False)

            if not final_thesis:
                msg = "Failed to generate the final thesis."
                if logger:
                    logger.log("EvaluatorOptimizerAgent", "System", msg, level="error")
                return msg

            if logger:
                logger.log("EvaluatorOptimizerAgent", "System", "Final polished thesis generated successfully.",
                           payload={"final_thesis": final_thesis[:500]})
            print("\n--- Final Thesis ---")
            print(final_thesis)

            return final_thesis

        except Exception as e:
            error_details = traceback.format_exc()
            if logger:
                logger.log("EvaluatorOptimizerAgent", "System",
                           f"Unhandled exception in evaluator-optimizer pipeline: {e}",
                           level="error",
                           traceback=error_details)
            print(f"Unhandled exception: {e}")
            return f"Unhandled exception: {e}"



## Analysis Function

In [309]:
def run_analysis(symbol: str):
    """Runs the full agentic analysis for a given stock symbol."""
    
    # Load API keys and configure Gemini
    load_env()
    genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

    # 1. Initialize Agents
    toolbox = ToolboxAgent()
    memory = MemoryAgent()
    planner = PlanningAgent()
    prompt_chainer = PromptChainingAgent()
    router = RoutingAgent()
    evaluator = EvaluatorOptimizerAgent()

    # 2. Define State
    state = {
        "symbol": symbol,
        "plan": [],
        "raw_data": {},
        "processed_news": [],
        "conversational_logs": [],
        "final_thesis": None
    }

    print(f"--- Starting Analysis for {symbol} ---")

    # 3. Establish Flow
    # Input Symbol -> Memory Agent -> Planning Engine Agent
    retrieved_memory = memory.retrieve(symbol, state)
    if retrieved_memory:
        print(f"\n--- Retrieved Memory for {symbol} ---")
        print(json.dumps(retrieved_memory, indent=4))
    
    state["plan"] = planner.generate_plan(symbol, state, json.dumps(retrieved_memory) if retrieved_memory else None)
    if not state["plan"]:
        print("Could not generate a plan. Exiting.")
        return

    print(f"\n--- Generated Plan for {symbol} ---")
    for step in state["plan"]:
        print(f"- {step}")

    # The sequence then calls the Toolbox Agent multiple times (Uncomment from here)
    for step in state["plan"]:
        if ("assessment" in step.lower() or "analysis" in step.lower()) and 'yfinance' not in state["raw_data"]:
            state["raw_data"]['yfinance'] = toolbox.fetch('yfinance', symbol, state)
        if ("news" in step.lower() or "finding" in step.lower() or "analysis" in step.lower()) and 'news' not in state["raw_data"]:
            state["raw_data"]['news'] = toolbox.fetch('newsapi', symbol, state)
        if ("economic" in step.lower() or "advancements" in step.lower()) and 'fred_gdp' not in state["raw_data"]:
            # A more robust implementation would parse the indicator
            state["raw_data"]['fred_gdp'] = toolbox.fetch('fred', 'GDP', state)
        if ("valuation" in step.lower() or "risk" in step.lower() or "report" in step.lower()) and "secEdgar" not in state["raw_data"]:
            # A more robust implementation would parse the indicator
            state["raw_data"]['secEdgar'] = toolbox.fetch('secEdgar', symbol, state)

    print("\n--- Fetched Raw Data ---")
    # Abridged printing for brevity
    if 'yfinance' in state["raw_data"]:
        print("  - Yahoo Finance data retrieved.")
    if 'news' in state["raw_data"]:
        print("  - News data retrieved.")
    if 'fred_gdp' in state["raw_data"]:
        print("  - FRED GDP data retrieved.")
    if 'secEdgar' in state["raw_data"]:
        print("  - Sec Edgar data retrieved.")

    # Toolbox Output -> Prompt Chaining Agent -> Routing Agent
    if 'news' in state["raw_data"] and state["raw_data"]['news']!=None and state["raw_data"]['news']['articles']:
        for article in state["raw_data"]['news']['articles']:
            processed_article = prompt_chainer.run(article['title'] + "\n" + article.get('description', ''), state)
            state["processed_news"].append(processed_article)
            
            state["classification"] = router.route(processed_article.get('classification', ''), state)

            print(f"\n--- Routing for article: '{article['title']}' ---")
            print(f"  - Classification: {processed_article.get('classification')}")
            print(f"  - Route: {state["classification"]}")

            # Routing -> Execution of Specialized Model (Placeholder)
            if state["classification"] == 'EarningsModelRun':
                print("  - (Placeholder) Would run a discounted cash flow model here.")
            elif state["classification"] == 'ComplianceCheck':
                print("  - (Placeholder) Would run a regulatory impact model here.")
            else:
                print("  - (Placeholder) Would run a general analysis model here.")

    # All data -> Evaluator–Optimizer Agent
    print("\n--- Generating Final Thesis with Evaluator-Optimizer ---")

    # Collect all relevant structured data for evaluation
    evaluator_data = {
        "symbol": state.get("symbol"),
        "classification": state.get("classification"),
        "financials": state.get("raw_data", {}).get("yfinance", []),
        "news": state.get("processed_news"),
        "economics": state.get("raw_data", {}).get("fred_gdp", []),
        "filings": state.get("raw_data", {}).get("secEdgar", [])
    }

    state["final_thesis"] = evaluator.run(evaluator_data, state)

    final_thesis = state["final_thesis"]
    logs = state.get("conversation_logs", [])

    evaluator = MultiAgentEvaluator()

    # LLM-based evaluation
    eval_result = evaluator.llm_grade(final_thesis)
    # Coordination metrics
    coordination = evaluator.coordination_efficiency(logs)

    # Basic heuristic parsing of scores from LLM text
    eval_text = eval_result.get("raw", "")
    clarity = factual = rigor = overall = 0

    # Regex patterns to extract scores
    patterns = {
        "clarity": r"clarity[:\s]*([0-9]+)\s*/\s*10",
        "accuracy": r"accuracy[:\s]*([0-9]+)\s*/\s*10",
        "rigor": r"rigor[:\s]*([0-9]+)\s*/\s*10",
        "overall": r"overall.*?([0-9]+)\s*/\s*10"
    }
    
    # Extract scores
    for key, pattern in patterns.items():
        match = re.search(pattern, eval_text, re.IGNORECASE)
        if match:
            score = int(match.group(1))
            if key == "clarity":
                clarity = score
            elif key == "accuracy":
                accuracy = score
            elif key == "rigor":
                rigor = score
            elif key == "overall":
                overall = score

    eval_metrics = {
        "clarity": clarity,
        "accuracy": accuracy,
        "rigor": rigor,
        "overall": overall,
        "source": eval_result.get("source", "unknown"),
        "evaluation_summary": eval_text,
    }

    state["evaluation"] = eval_metrics

    print("\n--- Evaluation Metrics ---")
    print(eval_metrics)

    memory.update(symbol, state["evaluation"])

    # Evaluator–Optimizer Output -> Memory Agent (Update)
    if state["final_thesis"]:
        # A more robust implementation would extract key metrics from the thesis
        memory.update(symbol, {"summary": state["final_thesis"]}, state)

        print(f"\n--- Completed Analysis for {symbol} ---")
        print("Final Thesis:")
        print(state["final_thesis"])
    
        return state


    return f"No summary generated for the symbol: {symbol}"

In [310]:
if __name__ == "__main__":
    # Prompt user for input with default value
    user_input = input("Enter stock symbol [default: NVDA]: ").strip()
    
    # Use NVDA if no input is provided
    symbol = user_input.upper() if user_input else "NVDA"
    
    run_analysis(symbol)

Environment variables loaded from config/aai_520_proj.config
--- Starting Analysis for NVDA ---

--- Retrieved Memory for NVDA ---
{
    "summary": "```json\n{\n  \"symbol\": \"NVDA\",\n  \"investment_thesis\": {\n    \"title\": \"NVIDIA: Dominating the AI Revolution - Initiating Coverage with a BUY Recommendation\",\n    \"executive_summary\": \"NVIDIA (NVDA) is a leading designer of graphics processing units (GPUs) and related software and services. The company's products are used in a variety of markets, including gaming, professional visualization, data centers, and automotive. Our investment thesis is predicated on NVIDIA's dominant position in the high-growth AI accelerator market, driven by increasing demand for generative AI, large language models, and accelerated computing across various industries. Despite inherent risks including competition and reliance on third party manufacturing, we believe NVIDIA's technological leadership, strong ecosystem, and strategic focus on high-

C:\Users\Dell\AppData\Local\Temp\ipykernel_24600\2234080774.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),



--- Generated Plan for NVDA ---
- 1. **Gather Recent News & Sentiment Analysis:** Use a news API (e.g., NewsAPI, Alpha Vantage) and sentiment analysis tools (e.g., VADER, TextBlob) to identify recent news articles and assess market sentiment surrounding NVDA. Focus on AI, data centers, and competition.
- 2. **Analyze Recent Earnings Transcripts:** Use a financial data provider (e.g., AlphaSense, FactSet) to access and analyze the most recent NVDA earnings call transcript. Identify key themes, management guidance, and analyst questions related to demand, supply chain, and competition.
- 3. **Competitor Analysis Update (AMD, INTC):** Refine the competitive landscape section by gathering recent financial data (revenue growth, market share) and product announcements from key competitors like AMD and Intel. Assess their progress in challenging NVIDIA's dominance in AI accelerators.
- 4. **Supply Chain Risk Assessment:** Investigate and update information on NVIDIA's supply chain, with a pa

UnboundLocalError: cannot access local variable 'accuracy' where it is not associated with a value